In [8]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.llms import Ollama
from crewai_tools import ScrapeWebsiteTool

In [12]:
duckgo = DuckDuckGoSearchRun()
sematic = SemanticScholarQueryRun()
wiki = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
website = ScrapeWebsiteTool(website_url='https://www.propertyhunter.com.my/sale/sabah/kota-kinabalu/one-sulaiman-platinum-tower-high-floor-fully-furnished/68487')

search_tools = [sematic, duckgo, wiki]

In [13]:
website.run()

Using Tool: Read website content


'One sulaiman|Platinum Tower|High Floor|Fully Furnished\nLog In\nProperties \nBuy\nRent\nNew Launches\nFind Agent\n News\nPropflix\nEvents\n1\nPH Conference 2024\nView Past Events\n More \nContact Us\nE-Book\nE-Mag\nStore\nArchive\nFind Job In Real Estate\nWomen in Real Estate\n Properties\nBuy\nRent\nNew\n Launches\nFind\n Agent\nNews\nPropflix\n Events\n1\nPH Conference 2024\nView Past Events\n More\nContact\n Us\nE-Book\nE-Mag\nStore\nArchive\nFind\n Job In Real Estate\nWomen\n in Real Estate\nLog In\nHELLO! FRIEND\nWelcome and start your journey with us.\nor fill up online form\n Sign\n Up\nLOG IN TO PROPERTY HUNTER\nor use your email account\nForgot\n your password?\nLog In\nVerification\n It seem like you have not verify your email address. If you have not receive your\n email verification, click the resend buttton below.\n Failed to send, please try again later.\n propertyhunter@gmail.com\n Resend Email Verification\n Please click on the link that has just been sent to your emai

In [3]:
ollama_llm = Ollama(model="openhermes")

In [4]:
researcher = Agent(
    role="Web Researcher",
    goal="Find information on web",
    backstory="People look for you to dig information on internet",
    verbose=True,
    allow_delegation=False,
    tools=[duckgo, website],
    llm=ollama_llm
)

In [5]:
qa_researcher = Agent(
    role="Information Verificaiton Web Researcher",
    goal="Verify information on web",
    backstory="People look for you to verify information on internet",
    verbose=True,
    allow_delegation=True,
    tools=[duckgo, website],
    llm=ollama_llm
)

In [6]:
task1 = Task(
    description="Collect me data for housing prices in Malaysia, when using DuckDuckGo, your action input should be following use keyword 'query' when using DuckDuckGo",
    expected_output="""Below are example for the following output structure: 
    '{'
        'name': 'One sulaiman|Platinum Tower|High Floor|Fully Furnished (ID: 068487)',
        'price': 'RM 518,000',
        'location': 'sepanggar, kota kinabalu, sabah',
        'type': 'condominium',
        'link': 'https://www.propertyhunter.com.my/sale/sabah/kota-kinabalu/one-sulaiman-platinum-tower-high-floor-fully-furnished/68487'
    '}'
    """,
    agent=researcher
)

In [7]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, qa_researcher],
    tasks=[task1],
    verbose=2
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [DEBUG]: == Working Agent: Web Researcher
 [INFO]: == Starting Task: Collect me data for housing prices in Malaysia, when using DuckDuckGo, your action input should be following use keyword 'query' when using DuckDuckGo


> Entering new CrewAgentExecutor chain...
Action: duckduckgo_search
Action Input: { "query": "housing prices in Malaysia" } 

No good DuckDuckGo Search Result was found

Thought: I should try searching for housing prices in specific websites.
Action: Search in a specific website
Action Input: { "search_query": "housing prices in Malaysia", "website": "propertyhunter.com.my" } 

I encountered an error while trying to use the tool. This was the error: 'OPENAI_API_KEY'.
 Tool Search in a specific website accepts these inputs: Search in a specific website(search_query: 'string', website: 'string') - A tool that can be used to semantic search a query from a specific URL content.

Thought: Let's start with DuckDuckGo search to get an idea of housing prices in Malaysia.
Act